In [4]:
import pandas as pd
from error_config import error_map

In [5]:
df_transformer = pd.read_csv('/home/sanjana/factual_evaluation_source_based/datasets/scored/dialogue_aggrefact_scored.csv')

df_llm = pd.read_csv('/home/sanjana/factual_evaluation_source_based/datasets/llm_annotations/sample/sample_annotations_sanjana.csv')
# df_llm = df_llm.sort_values('uuid').drop_duplicates(['docid'], keep='last')

len(df_llm)





172

In [6]:
df_transformer.head()

,Unnamed: 0.1,Unnamed: 0,DocID,Dialogue,Model,Summary,w/ Error,CorefE,CorefE_text,CircE,...,PredE_text,SubjObjE,SubjObjE_text,OtherE,OtherE_text,LinkE,LinkE_text,origin,SummaC-ZS_score,SummaC-Conv_score
0,0,0,13809941,Thelma: i dont have anything to wear\nLouisa: ...,human_ref,Louisa will lend Thelma her red velvet dress.,0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,FacEval,0.608673,0.224066
1,1,1,13809941,Thelma: i dont have anything to wear\nLouisa: ...,bart_large,Thelma doesn't have anything to wear. Louisa w...,0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,FacEval,0.271927,0.335007
2,2,2,13809941,Thelma: i dont have anything to wear\nLouisa: ...,mv-bart_large,Thelma doesn't have anything to wear. Louisa w...,0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,FacEval,0.271927,0.335007
3,3,3,13809941,Thelma: i dont have anything to wear\nLouisa: ...,co-ref bart large,Thelma doesn't have anything to wear. Louisa w...,0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,FacEval,0.090210,0.329921
4,4,4,13809941,Thelma: i dont have anything to wear\nLouisa: ...,condigsum bart large,Louisa will bring Thelma her red velvet dress.,0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,FacEval,0.374329,0.224066


In [7]:
# df_transformer['Dialogue'].values

In [8]:



qafact_kwargs = {"cuda_device": 0, "use_lerc_quip": True, \
        "verbose": True, "generation_batch_size": 32, \
        "answering_batch_size": 32, "lerc_batch_size": 8}

qafact_model_folder = "/home/sanjana/factual_evaluation_source_based/experiments/QAFactEval/models" # path to models downloaded with download_models.sh


class Evaluation():
    def __init__(self):
        return


    def get_questeval_scores(self, df, source_key, summary_key):
        from questeval.questeval_metric import QuestEval
        questeval = QuestEval(no_cuda=True)
        sources = list(df[source_key].values)
        summaries = list(df[summary_key].values)
        score = questeval.corpus_questeval(
            hypothesis=summaries, 
            sources=sources,
        )
        df['QuestEval_score'] = score['ex_level_scores']
        return df

    def get_qafacteval_scores(self, df, source_key, summary_key):
        from qafacteval import QAFactEval
        qafacteval = QAFactEval(
            lerc_quip_path=f"{qafact_model_folder}/quip-512-mocha",
            generation_model_path=f"{qafact_model_folder}/generation/model.tar.gz",
            answering_model_dir=f"{qafact_model_folder}/answering",
            lerc_model_path=f"{qafact_model_folder}/lerc/model.tar.gz",
            lerc_pretrained_model_path=f"{qafact_model_folder}/lerc/pretraining.tar.gz",
            **qafact_kwargs
        )
        sources = list(df[source_key].values)
        summaries = list(df[summary_key].values)
        summaries = [[summ] for summ in summaries]
        results = qafacteval.score_batch_qafacteval(sources, summaries, return_qa_pairs=True)
        scores = [ each[0]['qa-eval']['lerc_quip'] for each in results]
        df['QAFactEval_score'] = scores
        return df

    def get_summac_scores(self, df, source_key, summary_key):
        from summac.model_summac import SummaCZS, SummaCConv
        model_zs = SummaCZS(granularity="sentence", model_name="vitc", device="cuda") # If you have a GPU: switch to: device="cuda"
        model_conv = SummaCConv(models=["vitc"], bins='percentile', granularity="sentence", nli_labels="e", device="cuda", start_file="default", agg="mean")

        sources = list(df[source_key].values)
        summaries = list(df[summary_key].values)
        score_zs = model_zs.score(sources, summaries)
        score_conv = model_conv.score(sources, summaries)
        df['SummaC-ZS_score']= score_zs['scores']
        df['SummaC-Conv_score'] = score_conv['scores']
        return df

        
    

        
        

In [9]:
EvalMetrics = Evaluation()


In [ ]:
df_transformer = EvalMetrics.get_questeval_scores(df_transformer, 'Dialogue', 'Summary')

In [ ]:
df_transformer.head()

In [12]:
df_transformer.to_csv('/home/sanjana/factual_evaluation_source_based/datasets/scored/dialogue_aggrefact_scored.csv')

In [2]:
from questeval.questeval_metric import QuestEval
questeval = QuestEval(no_cuda=False)

source_1 = "Since 2000, the recipient of the Kate Greenaway medal has also been presented with the Colin Mears award to the value of 35000."
prediction_1 = "Since 2000, the winner of the Kate Greenaway medal has also been given to the Colin Mears award of the Kate Greenaway medal."
references_1 = [
    "Since 2000, the recipient of the Kate Greenaway Medal will also receive the Colin Mears Awad which worth 5000 pounds",
    "Since 2000, the recipient of the Kate Greenaway Medal has also been given the Colin Mears Award."
]

source_2 = "He is also a member of another Jungiery boyband 183 Club."
prediction_2 = "He also has another Jungiery Boyband 183 club."
references_2 = [
    "He's also a member of another Jungiery boyband, 183 Club.", 
    "He belonged to the Jungiery boyband 183 Club."
]

score = questeval.corpus_questeval(
    hypothesis=[prediction_1, prediction_2], 
    sources=[source_1, source_2],
    list_references=[references_1, references_2]
)

print(score)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'corpus_score': 0.6115375215309217, 'ex_level_scores': [0.569881930994609, 0.6531931120672343]}


In [8]:
from summac.model_summac import SummaCZS, SummaCConv

model_zs = SummaCZS(granularity="sentence", model_name="vitc", device="cuda") # If you have a GPU: switch to: device="cuda"
model_conv = SummaCConv(models=["vitc"], bins='percentile', granularity="sentence", nli_labels="e", device="cuda", start_file="default", agg="mean")

document = """Scientists are studying Mars to learn about the Red Planet and find landing sites for future missions.
One possible site, known as Arcadia Planitia, is covered instrange sinuous features.
The shapes could be signs that the area is actually made of glaciers, which are large masses of slow-moving ice.
Arcadia Planitia is in Mars' northern lowlands."""

summary1 = "There are strange shape patterns on Arcadia Planitia. The shapes could indicate the area might be made of glaciers. This makes Arcadia Planitia ideal for future missions."
score_zs1 = model_zs.score([document], [summary1])
score_conv1 = model_conv.score([document], [summary1])
print("[Summary 1] SummaCZS Score: %.3f; SummacConv score: %.3f" % (score_zs1["scores"][0], score_conv1["scores"][0])) # [Summary 1] SummaCZS Score: 0.582; SummacConv score: 0.536

summary2 = "There are strange shape patterns on Arcadia Planitia. The shapes could indicate the area might be made of glaciers."
score_zs2 = model_zs.score([document, document], [summary2, summary2])
score_conv2 = model_conv.score([document], [summary2])
print("[Summary 2] SummaCZS Score: %.3f; SummacConv score: %.3f" % (score_zs2["scores"][0], score_conv2["scores"][0])) # [Summary 2] SummaCZS Score: 0.877; SummacConv score: 0.709




<All keys matched successfully>
[Summary 1] SummaCZS Score: 0.583; SummacConv score: 0.536
[Summary 2] SummaCZS Score: 0.877; SummacConv score: 0.709


In [1]:
from qafacteval import QAFactEval
kwargs = {"cuda_device": 0, "use_lerc_quip": True, \
        "verbose": True, "generation_batch_size": 32, \
        "answering_batch_size": 32, "lerc_batch_size": 8}

model_folder = "/home/sanjana/factual_evaluation_source_based/experiments/QAFactEval/models" # path to models downloaded with download_models.sh
metric = QAFactEval(
    lerc_quip_path=f"{model_folder}/quip-512-mocha",
    generation_model_path=f"{model_folder}/generation/model.tar.gz",
    answering_model_dir=f"{model_folder}/answering",
    lerc_model_path=f"{model_folder}/lerc/model.tar.gz",
    lerc_pretrained_model_path=f"{model_folder}/lerc/pretraining.tar.gz",
    **kwargs
)

results = metric.score_batch_qafacteval(["This is a source document", "This is a source document"], [["This is a summary."], ["This is a summary."]], return_qa_pairs=True)
score = results[0][0]['qa-eval']['lerc_quip']


2023-08-30 20:07:04.043743: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-30 20:07:05.709060: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


RuntimeError: Error(s) in loading state_dict for _QuestionGenerationModel:
	Missing key(s) in state_dict: "bart.lm_head.weight". 